In [ ]:
# 14 April 2023
# nrobot
# acquire urls 

import pandas as pd
from pandarallel import pandarallel
from collections import Counter
pandarallel.initialize(progress_bar=True)
import os
import datetime
import tldextract   

In [ ]:
from urlextract import URLExtract
from urllib.parse import urlparse
import os


In [ ]:
# CODE SOURCE: from parse_links import parse_domains
# TODO: consider replacing urlparse with tldextract ?
# TODO: push check_dns flag back to original src file parse_links.py

extractor = URLExtract()
extractor.add_enclosure("(", ")")
extractor.add_enclosure("[", "]")

def parse_domain(url):
    if "//" not in url:
        url = "//" + url
    return urlparse(url).netloc

def parse_domains(text):
    return list(set(parse_domain(url) for url in extractor.gen_urls(text)))

# TODO: refactor the below
def parse_domains_and_validate(text):
    return list(set(parse_domain(url) for url in extractor.gen_urls(text, check_dns=True)))

In [ ]:
file = './nogit_data/list_of_post_contents.csv'
df = pd.read_csv(file)#, nrows=10_00)

In [ ]:
# TODO: decorator that annotates filename with pandas version
def retrieve_domains(df, outfile='nogit_data/domains.pkl', overwrite_existing=False):
    # df - must contain 'post_text' column

    # if the file exists, load it.  # if overwrite, then re-run and save it.
    # if file not exists, run and save it.
    # return file
    
    if (not os.path.exists(outfile)) or (os.path.exists(outfile) & overwrite_existing):
        domains = df['post_text'].dropna().parallel_apply(parse_domains)
        domains[domains.apply(bool)].to_pickle(outfile)
    else: 
        domains = pd.read_pickle(outfile)
    
    # TODO: these 3 lines can definitely be one line. 
    t = os.path.getmtime(outfile)
    t = datetime.datetime.fromtimestamp(t)
    t = t.strftime('%H:%M:%S / %B %d %Y')
    print(f'Loaded {outfile}, modified at {t}')
    return domains

def retrieve_domains_and_validate(df, outfile='nogit_data/domains_validated_only.pkl', overwrite_existing=False):
    # same as above, but with check_dns = True on the parse_domain function
    
    if (not os.path.exists(outfile)) or (os.path.exists(outfile) & overwrite_existing):
        domains = df['post_text'].dropna().parallel_apply(parse_domains_and_validate)
        domains[domains.apply(bool)].to_pickle(outfile)
    else: 
        domains = pd.read_pickle(outfile)
    return domains

In [ ]:
domains = retrieve_domains(df)#, overwrite_existing=True)

In [ ]:
domains_validated_only = retrieve_domains_and_validate(df)

In [ ]:
display(domains.sample(n=40))
domains_validated_only.sample(n=40)

In [ ]:
# TODO - fix url parsing, bug example:
print(domains.loc[111371])
print(domains_validated_only.loc[111371])
#print(df.iloc[111371].post_text)

In [ ]:
def count_urls(df):
    # Input: df where each row is a python list of urls

    cnt = Counter()
    all_links = []
    for link_list in df.values:
        all_links.extend(link_list)
    for link in all_links: 
        cnt[link] += 1

    df_links = pd.DataFrame.from_dict(cnt, orient='index').reset_index()
    df_links.columns = ['url', 'counts']
    df_links['normalized_counts'] = df_links['counts'] / sum(df_links['counts']) * 100
    return df_links

df_links = count_urls(domains)
display(df_links.sort_values(by='counts', ascending=False))
df_links.sort_values(by='counts').iloc[:20]

In [ ]:
# apply to df_links, ? then count again?  

In [ ]:
def extract_domain(list_url):
    return ['.'.join(tldextract.extract(url)[1:]) for url in list_url]
no_suffix = domains.apply(extract_domain)
display(no_suffix)
tmp = count_urls(no_suffix)
tmp
# -- 
no_suffix = domains_validated_only.apply(extract_domain)
display(no_suffix)
tmp = count_urls(no_suffix)
tmp

In [ ]:
display(tmp.sort_values(by='counts').iloc[:20])
display(tmp.sort_values(by='counts').iloc[-20:])

In [ ]:
import altair as alt
#tmp.sort_values(by='counts').iloc[-20:].counts.plot(kind='bar', xlabel='url')
_tmp = count_urls(no_suffix)
top20 = _tmp.sort_values(by='counts').iloc[-20:]
alt.Chart(top20).mark_bar().encode(
    x=alt.X('url', axis=alt.Axis(labelAngle=-45)),
    y='counts:Q',
    text='url'
).properties(width=600, height=200)